In [ ]:
import pandas as pd
import numpy as np
import json
import time
import requests

import threading


In [ ]:
def get_category(query_param,base_url='https://www.googleapis.com/books/v1/volumes?q=',fields='&fields=items(volumeInfo(categories))'):
    cat=[]
    req=requests.get(base_url+query_param+fields)
    print(req.status_code)
    if req.status_code==200:
        result=json.loads(req.text)
        try:
            cat=result['items'][0]['volumeInfo']['categories']
        except KeyError:
            return cat
    elif req.status_code==503:
        time.sleep(3)
    return cat

In [ ]:
def write_dict_to_csv(filename, cat_dict, orient="index", index=False):
    df = pd.DataFrame.from_dict(cat_dict, orient=orient, columns=["Genre"])
    df.to_csv(filename, index=index, index_label="ISBN")


In [ ]:
exitFlag = 0
class myThread (threading.Thread):
   def __init__(self, threadID, r, steps):
      threading.Thread.__init__(self)
      self.threadID = threadID
      self.r = r
      self.steps = steps

   def run(self):
      print("Starting " + str(self.r )+ "to" + str(self.r+steps))
      get_categories(self.r,  self.steps)
      print("Exiting " + str(self.r) + "to" + str(self.r+steps))


In [ ]:
common_data=pd.read_csv('books_common_data.csv')
unique_isbns=list(set(common_data.ISBN))


In [ ]:
extracted_data = pd.read_csv('first_7k_books.csv')
extracted_ISBNs = list(set(extracted_data.ISBN))


In [ ]:
non_extracted_ISBNs = list(set(unique_isbns) - set(extracted_ISBNs))

There are around 24684 books

In [ ]:
def get_categories(start, steps):
    cat_dict={}
    for i in range(start, start+steps):
        print(i)
        try:
            cat_dict[non_extracted_ISBNs[i]]=get_category(base_url='https://www.googleapis.com/books/v1/volumes?q=',query_param=non_extracted_ISBNs[i],fields='&fields=items(volumeInfo(categories))')
        except:
            time.sleep(3)
            try:
                cat_dict[non_extracted_ISBNs[i]]=get_category(base_url='https://www.googleapis.com/books/v1/volumes?q=',query_param=non_extracted_ISBNs[i],fields='&fields=items(volumeInfo(categories))')
            except:
                continue
    write_dict_to_csv('Genres_'+str(start)+'_' +
                      str(start+steps), cat_dict, index=True)
    cat_dict={}


In [ ]:
cat_dict={}
steps= 5
ranges = [0, 10]  # AMEL
# ranges = [] 5000 -10000 # UKESH
# ranges = []  10000 - 15000 # KAMAL
# ranges = [] # SRIKAANTH
for r in range(ranges[0], ranges[1], steps):
    thread = myThread(r, r, steps)
    thread.start()
